In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
artworks="/kaggle/input/best-artworks-of-all-time"
imagenet="/kaggle/input/imagenet"

In [6]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

2024-05-01 10:34:51.686318: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 10:34:51.686484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 10:34:51.825650: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow version: 2.15.0


In [8]:
# Step 3: Implementing the Neural Style Transfer (NST) model

# Import necessary modules
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

# Define the NST model function
def build_nst_model():
    # Load the pre-trained VGG16 model (without the top layers)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Choose intermediate layers for content and style representations
    content_layers = ['block4_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
    
    # Output the content and style representations
    content_outputs = [base_model.get_layer(layer).output for layer in content_layers]
    style_outputs = [base_model.get_layer(layer).output for layer in style_layers]
    
    # Combine content and style outputs
    model_outputs = content_outputs + style_outputs
    
    # Create the model
    return Model(inputs=base_model.input, outputs=model_outputs)

# Build the NST model
nst_model = build_nst_model()
nst_model.summary()


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,995,072 (38.13 MB)

 Trainable params: 9,995,072 (38.13 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Step 4: Training the Neural Style Transfer (NST) model

# Define loss functions
def content_loss(content, target):
    return K.mean(K.square(content - target))

def gram_matrix(input_tensor):
    channels = int(input_tensor.shape[-1])
    a = K.reshape(input_tensor, (-1, channels))
    n = K.shape(a)[0]
    gram = K.dot(a, K.transpose(a)) / tf.cast(n, tf.float32)
    return gram

def style_loss(style, target):
    style_gram = gram_matrix(style)
    target_gram = gram_matrix(target)
    return K.mean(K.square(style_gram - target_gram))

# Define total variation loss to maintain spatial coherence
def total_variation_loss(x):
    a = K.square(x[:, :-1, :-1, :] - x[:, 1:, :-1, :])
    b = K.square(x[:, :-1, :-1, :] - x[:, :-1, 1:, :])
    return K.mean(K.pow(a + b, 1.25))

# Define hyperparameters
content_weight = 0.025
style_weight = 1.0
total_variation_weight = 1e-6

# Define optimizer
optimizer = Adam(learning_rate=0.02)

# Define function to compute total loss
def compute_loss(model, loss_weights, init_image, content_target, style_target):
    style_outputs = model(style_target)
    print(style_outputs.shape)
    content_outputs = model(content_target)
    
    style_loss_value = 0
    content_loss_value = 0

    content_weight, style_weight = loss_weights
    
    # Compute content loss
    for target_content, comb_content in zip(content_target, content_outputs):
        content_loss_value += content_weight * content_loss(target_content, comb_content)
    
    # Compute style loss
    for target_style, comb_style in zip(style_target, style_outputs):
        style_loss_value += style_weight * style_loss(target_style, comb_style)
    
    # Compute total variation loss
    total_variation_loss_value = total_variation_weight * total_variation_loss(init_image)
    
    # Total loss
    total_loss = content_loss_value + style_loss_value + total_variation_loss_value
    return total_loss, style_loss_value, content_loss_value, total_variation_loss_value

# Define function to compute gradients and apply them using an optimizer
@tf.function()
def compute_grads(cfg):
    with tf.GradientTape() as tape:
        all_loss = compute_loss(**cfg)
    grads = tape.gradient(all_loss[0], cfg['init_image'])
    optimizer.apply_gradients([(grads, cfg['init_image'])])
    return all_loss

# Training loop
def train_nst(model, content_image, style_image, epochs=10, steps_per_epoch=100):
    # Initialize generated image as content image
    init_image = tf.Variable(content_image)
    
    # Define target content and style representations
    content_target = model(content_image)
    style_target = model(style_image)
    
    # Define configuration for loss computation
    cfg = {
        'model': model,
        'loss_weights': (content_weight, style_weight),
        'init_image': init_image,
        'content_target': content_target,
        'style_target': style_target
    }
    
    # Training loop
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        for step in range(steps_per_epoch):
            loss_values = compute_grads(cfg)
            if step % 50 == 0:
                print(f"Step {step + 1}, Total Loss: {loss_values[0]}")
    
    return init_image

style_image_path = "/kaggle/input/best-artworks-of-all-time/images/images/Francisco_Goya/Francisco_Goya_1.jpg"
style_image = keras.preprocessing.image.load_img(style_image_path, target_size=(224, 224))
style_image = keras.preprocessing.image.img_to_array(style_image)
style_image = np.expand_dims(style_image, axis=0)
style_image = keras.applications.vgg16.preprocess_input(style_image)

content_image_path ='/kaggle/input/imagenet/imagenet/train/ILSVRC2012_val_00005001.JPEG'
content_image = keras.preprocessing.image.load_img(content_image_path, target_size=(224, 224))
content_image = keras.preprocessing.image.img_to_array(content_image)
content_image = np.expand_dims(content_image, axis=0)
content_image = keras.applications.vgg16.preprocess_input(content_image)
# Build the NST model
nst_model = build_nst_model()

# Convert datasets to TensorFlow tensors
content_image = tf.convert_to_tensor(content_image)  # Replace with your content image
style_image = tf.convert_to_tensor(style_image)  # Replace with your style image

# Train the NST model
generated_image = train_nst(nst_model, content_image, style_image, epochs=5, steps_per_epoch=100)


Epoch 1/5


ValueError: in user code:

    File "/tmp/ipykernel_34/7038979.py", line 63, in compute_grads  *
        all_loss = compute_loss(**cfg)
    File "/tmp/ipykernel_34/2818350052.py", line 35, in compute_loss  *
        style_outputs = model(style_target)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.10/site-packages/keras/src/layers/input_spec.py", line 156, in assert_input_compatibility
        raise ValueError(

    ValueError: Layer 'functional_9' expected 1 input(s). Received 6 instead.


In [18]:
style_image.shape

TensorShape([1, 224, 224, 3])

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, optimizers
import numpy as np
import os

# Function to preprocess images
def preprocess_image(image_path):
    img = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.vgg16.preprocess_input(img)
    return img

# Function to generate stylized images using NST model
def generate_stylized_images(nst_model, content_images, style_image):
    stylized_images = []
    for content_image in content_images:
        generated_image = nst_model(tf.constant(content_image), tf.constant(style_image))
        stylized_images.append(generated_image)
    return stylized_images

# Function to create a linear classifier model
def create_linear_classifier(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Flatten()(inputs)
    x = layers.Dense(256, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

# Load the pretrained VGG16 model
base_model = keras.applications.VGG16(weights='imagenet', include_top=False)

# Define the NST model
def build_nst_model():
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
    content_layers = ['block4_conv2']

    inputs = layers.Input(shape=(224, 224, 3))
    x = inputs

    style_outputs = []
    content_outputs = []

    for layer in base_model.layers:
        x = layer(x)
        if layer.name in style_layers:
            style_outputs.append(x)
        if layer.name in content_layers:
            content_outputs.append(x)

    model_outputs = style_outputs + content_outputs
    return Model(inputs, model_outputs)

# Load ImageNet dataset
imagenet_dir = '/kaggle/input/imagenet/imagenet/train/'
image_files = os.listdir(imagenet_dir)
content_images = [preprocess_image(os.path.join(imagenet_dir, file)) for file in image_files]

# Load style image
style_image_path = "/kaggle/input/best-artworks-of-all-time/images/images/Francisco_Goya/Francisco_Goya_1.jpg"
style_image = preprocess_image(style_image_path)

# Generate stylized images using NST model
nst_model = build_nst_model()
stylized_images = generate_stylized_images(nst_model, content_images, style_image)

# Define labels for the images (example: for ImageNet, labels can be the image categories)
labels = [0] * len(content_images)

# Concatenate the stylized images and original images
all_images = np.concatenate((content_images, stylized_images), axis=0)
labels.extend([1] * len(stylized_images))

# Shuffle the data
idx = np.random.permutation(len(all_images))
all_images = np.array(all_images)[idx]
labels = np.array(labels)[idx]

# Split the dataset into training and testing sets
split_idx = int(0.8 * len(all_images))
train_images, test_images = all_images[:split_idx], all_images[split_idx:]
train_labels, test_labels = labels[:split_idx], labels[split_idx:]

# Create linear classifier model
linear_classifier = create_linear_classifier((7, 7, 512), 2)

# Compile and train the model
linear_classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
linear_classifier.fit(train_images, train_labels, epochs=5, batch_size=32, validation_data=(test_images, test_labels))


TypeError: too many positional arguments